In [2]:
#File name: Train Network
#Names of all members of the group: James Kaufman, Eamon Kostopulos, 
#Ahmed Mohammed, Sebastian Cortes, and Jonathan Goral 
#Project name and description: DL Music Classification - A neural network that classifies music genres
#Any special execution instruction: librosa, numpy, pydub, os, pickle, pylab, ffmeg, glob, matplotlib.pyplot
#Replaced By: SmallDenseNetwork
#Date: 12/4/18
import librosa
import numpy as np
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.utils.np_utils import to_categorical
from keras.preprocessing import sequence
from keras import initializers

import librosa.feature
import librosa.display
import os
import pickle

Using TensorFlow backend.


In [23]:
#code taken from https://blog.manash.me/building-a-dead-simple-word-recognition-engine-using-convnet-in-keras-25e72c19c12b
def encodeGenres(genrelist):
    uniq_ids, row_ids = np.unique(genrelist, return_inverse = True)
    row_ids = row_ids.astype(np.int32, copy = False)
    hot_labels = to_categorical(row_ids, len(uniq_ids))
    return hot_labels

In [26]:
genre_files = os.listdir("PklFiles")
x_train = []
y_train = []
x_test = []
y_test = []

#load the data while evenly splitting each genre into test and train

test_ratio = .8

for file in genre_files:
    genre = file[:-4]
    with open("PklFiles/" + file, 'rb') as f:
        temp = pickle.load(f)
        train_count = int(len(temp)*test_ratio)
        count = 0
        for data in temp:
            if count < train_count:
                x_train.append(data)
                y_train.append(genre)
            else:
                x_test.append(data)
                y_test.append(genre)
            count+=1

#convert the lists to numpy arrays
x_train = np.asarray(x_train)
x_test = np.asarray(x_test)

#reshape to allow it to be used on the 2D CNN
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)

y_train = encodeGenres(y_train)
y_test = encodeGenres(y_test)

print("Shape of x_test: " + str(x_test.shape))
print("Shape of y_test: " + str(y_test.shape))
print("Shape of x_train: " + str(x_train.shape))
print("Shape of y_train: " + str(y_train.shape))


Shape of x_test: (9594, 20, 157, 1)
Shape of y_test: (9594, 8)
Shape of x_train: (38373, 20, 157, 1)
Shape of y_train: (38373, 8)


In [30]:
#train the neural network

#maybe randomize it?

#decrease 2D size layer as we continue to pool
model = Sequential()
model.add(Conv2D(256, kernel_size = (2,2), kernel_initializer='glorot_uniform', input_shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3])))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, kernel_size = (2,2), kernel_initializer='glorot_uniform'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size = (2,2), kernel_initializer='glorot_uniform'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(8))
model.add(Activation('elu'))

In [31]:
#compile our model from training
model.compile(optimizer = 'RMSProp', loss = 'categorical_crossentropy', metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 19, 156, 256)      1280      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 78, 256)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 78, 256)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 77, 128)        131200    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 38, 128)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 37, 64)         32832     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 1, 18, 64)         0         
__________

In [32]:
#fit our model as we cointiue to cycle through epochs while pritning out information
model.fit(x_train, y_train, epochs = 50, batch_size = 16, validation_data=(x_test, y_test), verbose = 1)
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 38373 samples, validate on 9594 samples
Epoch 1/50
  640/38373 [..............................] - ETA: 19:09 - loss: 2.3701 - acc: 0.1422

KeyboardInterrupt: 